In [214]:
from math import sqrt
import numpy as np
import unicodedata
import random
import string

# Tarea 1: Exponenciación rápida

### a)

In [28]:
def e(a,b):
    if b==0:
        return 1
    if b>0 and b%2==0:
        return e(a*a,b/2)
    if b>0 and b%2==1:
        return e(a*a,(b-1)/2)*a

In [32]:
print(e(2,3),e(0,0))

8 1


### b)

In [4]:
def e(a,b):
    if b==0:
        return 1
    if b%2==0:
        return e(a*a,b/2)
    if b>0 and b%2==1:
        return e(a*a,(b-1)/2)*a
    if b<0 and b%2==1:
        return e(a*a,(b+1)/2)/a

In [17]:
print(e(-3,-2),e(2,-2),e(-3,-1))

0.1111111111111111 0.25 -0.3333333333333333
8 1


### c)

In [32]:
def e(a,b,n):
    if b==0:
        return 1
    if b>0 and b%2==0:
        return e(a*a%n,b/2,n)
    if b>0 and b%2==1:
        return e(a*a%n,(b-1)/2,n)*a%n
    ### TODO: cuando b es negativo, podríamos elevar el inverso, ¿y si no existe inverso?
    if b<0 b%2==0:
        return e(a*a,b/2)
    if b<0 and b%2==1:
        return e(a*a%n,(b+1)/2,n)/a

In [40]:
e(-3,-2,6)

0.3333333333333333

# Tarea 2: Criptosistema de Vigenère

In [137]:
def normalize(c):
    return unicodedata.normalize("NFD",c)[0]

def encode(cadena):
    cadena=cadena.upper().replace('Ñ','GN')
    cadena=''.join(normalize(c) for c in cadena if c.isalnum())
    return cadena

In [292]:
class Vigenere:
    
    def __init__(self,alphabet,key=None,keyLen=None):
        self.A=alphabet
        self.n=len(alphabet)
        
        self.value=dict(zip(self.A,range(self.n)))
        self.character={v:k for k,v in self.value.items()}
        
        if key==None:
            if keyLen==None:
                keyLen=10
            self.key=''.join(random.choice(self.A) for _ in range(keyLen))
        else:
            self.key=key
            
    def getValue(self,c):
        return self.value[c]
    
    def getCharacter(self,v):
        return self.character[v]
            
    def keyToLength(self, length):
        return (self.key * (int(length/len(self.key))+1))[:length]
            
    def cipher(self,msg):
        k=self.keyToLength(len(msg))
        msg_values=list(map(self.getValue,msg))
        key_values=list(map(self.getValue,k))
        result=[(msg_values[i]+key_values[i])%self.n for i in range(len(msg))]
        return ''.join((map(self.getCharacter,result)))
        
    def decipher(self, msg):
        k=self.keyToLength(len(msg))
        msg_values=list(map(self.getValue,msg))
        key_values=list(map(self.getValue,k))
        result=[(msg_values[i]-key_values[i])%self.n for i in range(len(msg))]
        return ''.join((map(self.getCharacter,result)))

In [162]:
alphabet=string.ascii_uppercase+string.digits

In [293]:
#vig = Vigenere(alphabet,'CLAVEMUYSECRETA')
vig = Vigenere(alphabet)

In [294]:
msg=encode('el secreto se protege a sí mismo. Mi constraseña del WI-FI es 123456')
msg

'ELSECRETOSEPROTEGEASIMISMOMICONSTRASEGNADELWIFIES123456'

In [295]:
cifrado=vig.cipher(msg)
cifrado

'FNLPD431ZXFRKZUR5MLXJOB3N1BQNTOUM2B53OYFEGE7JS7M3635XG7'

In [296]:
vig.decipher(cifrado)

'ELSECRETOSEPROTEGEASIMISMOMICONSTRASEGNADELWIFIES123456'

# Tareas 3 y 4: Rompiendo el criptosistema de Vigenère

Vamos a poner un ejemplo del ataque al criptosistema de Vigenère usando el texto del ejercicio 4.
El ataque se basa en la obtención de la clave por el siguiente procedimiento:
- Primero se averigua su longitud, TODO
Nos aprovechamos de que la clave se repite hasta ocupar la longitud del texto a cifrar
https://github.com/ichantzaras/creamcrackerz

In [422]:
text="""UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWO
FFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZIVQWOFROGTT
NNXEVIVIBPDTTGAHVIACLAYKGJIEQHGECMESNNOCTHSGGNVWTQHKBPR
HMVUOYWLIAFIRIGDBOEBQLIGWARQHNLOISQKEPEIDVXXNETPAXNZGDX
WWEYQCTIGONNGJVHSQGEATHSYGSDVVOAQCXLHSPQMDMETRTMDUXTEQQ
JMFAEEAAIMEZREGIMUECICBXRVQRSMENNWTXTNSRNBPZHMRVRDYNECG
SPMEAVTENXKEQKCTTHSPCMQQHSQGTXMFPBGLWQZRBOEIZHQHGRTOBSG
TATTZRNFOSMLEDWESIWDRNAPBFOFHEGIXLFVOGUZLNUSRCRAZGZRTTA
YFEHKHMCQNTZLENPUCKBAYCICUBNRPCXIWEYCSIMFPRUTPLXSYCBGCC
UYCQJMWIEKGTUBRHVATTLEKVACBXQHGPDZEANNTJZTDRNSDTFEVPDXK
TMVNAIQMUQNOHKKOAQMTBKOFSUTUXPRTMXBXNPCLRCEAEOIAWGGVVUS
GIOEWLIQFOZKSPVMEBLOHLXDVCYSMGOPJEFCXMRUIGDXNCCRPMLCEWT
PZMOQQSAWLPHPTDAWEYJOGQSOAVERCTNQQEAVTUGKLJAXMRTGTIEAFW
PTZYIPKESMEAFCGJILSBPLDABNFVRJUXNGQSWIUIGWAAMLDRNNPDXGN
PTTGLUHUOBMXSPQNDKBDBTEECLECGRDPTYBVRDATQHKQJMKEFROCLXN
FKNSCWANNAHXTRGKCJTTRRUEMQZEAEIPAWEYPAJBBLHUEHMVUNFRPVM
EDWEKMHRREOGZBDBROGCGANIUYIBNZQVXTGORUUCUTNBOEIZHEFWNBI
GOZGTGWXNRHERBHPHGSIWXNPQMJVBCNEIDVVOAGLPONAPWYPXKEFKOC
MQTRTIDZBNQKCPLTTNOBXMGLNRRDNNNQKDPLTLNSUTAXMNPTXMGEZKA
EIKAGQ"""
text=encode(text)

Primero encontramos las sequencias de caracteres que se repiten en el texto

In [423]:
# Encuentra secuencias de caracteres que se repiten dos o más veces, así como las posiciones en las que aparecen
def repeated_seq_pos(text, seq_len):
    seq_pos = {}  # Guardaremos en un diccionario todas las secuencias y las posiciones en las que aparecen
    for i, char in enumerate(text): # Recorremos el texto
        next_seq = text[i:i+seq_len] # Tomamos cada posible secuencia de seq_len caracteres
        if next_seq in seq_pos.keys(): # Si la secuencia ya había ocurrido, añadimos la nueva posición
            seq_pos[next_seq].append(i)
        else: # Si no había ocurrido, añadimos su primera ocurrencia
            seq_pos[next_seq] = [i]
    repeated = list(filter(lambda x: len(seq_pos[x]) >= 2, seq_pos)) # Nos quedamos con las secuencias que tienen 2 o más ocurrencias
    rep_seq_pos = [(seq, seq_pos[seq]) for seq in repeated]
    return rep_seq_pos

In [424]:
secuencias=repeated_seq_pos(text,3)
secuencias[:4]

[('UEC', [0, 292]),
 ('VAC', [11, 518]),
 ('GEZ', [18, 1040]),
 ('QMD', [21, 259])]

Ahora calculamos las diferencias (el espacio) entre las posiciones de las ocurrencias.
Si una secuencia se repite, es probable que lo haga tras un múltiplo de la longitud de la clave.

In [425]:
# Devuelve una lista con las diferencias entre las posiciones de las ocurrencias
def get_spacings(positions):
    return [positions[i+1] - positions[i] for i in range(len(positions)-1)]

In [426]:
seq_espacios = {}
for seq, positions in secuencias:
    seq_espacios[seq] = get_spacings(positions)
[(k,v) for k,v in seq_espacios.items()][:4]

[('UEC', [292]), ('VAC', [507]), ('GEZ', [1022]), ('QMD', [238])]

Ahora hallamos los divisores propios de esas diferencias.

In [427]:
def get_factors(number):
    factors = set()
    for i in range(1, int(sqrt(number))+1):
        if number % i == 0:
            factors.add(i)
            factors.add(number//i)
    return sorted(factors)

In [428]:
factor_lists = []
for espacios in seq_espacios.values():
    for espacio in espacios:
        factor_lists.append(get_factors(number=espacio))
factor_lists[:4]

[[1, 2, 4, 73, 146, 292],
 [1, 3, 13, 39, 169, 507],
 [1, 2, 7, 14, 73, 146, 511, 1022],
 [1, 2, 7, 14, 17, 34, 119, 238]]

Los candidatos a ser la longitud de clave dividiran a la mayoría de los espacios entre ocurrencias de cada secuencia

In [429]:
def candidate_key_lengths(factor_lists, max_key_len):
    # Unimos todos los factores en una lista
    all_factors = [factor_lists[lst][fac] for lst in range(len(factor_lists)) for fac in range(len(factor_lists[lst]))]
    # Excluimos los mayores que la longitud máxima de clave
    candidate_lengths = list(filter(lambda x:  x <= max_key_len, all_factors))
    # Los ordenamos según el número de veces que aparecen
    sorted_candidates = sorted(set(candidate_lengths), key=lambda x: all_factors.count(x), reverse=True)
    return sorted_candidates

In [430]:
ckl = candidate_key_lengths(factor_lists=factor_lists, max_key_len=20)
ckl

[1, 7, 3, 2, 14, 5, 4, 9, 6, 13, 15, 8, 17, 10, 11, 12, 16, 19, 20, 18]

Obviamente el 1 es el factor que más se repite, el 7 es nuestro principal candidato a ser la longitud de la clave.

Dividimos el texto en bloques de longitud 7.

In [431]:
keyLen=7

In [432]:
def get_blocks(text, size):
    blocks = [text[i:i+size] for i in range(0, len(text)-size, size)]
    return blocks

In [433]:
blocks=get_blocks(text,keyLen)
print(blocks)

['UECWKDV', 'LOTTVAC', 'KTPVGEZ', 'QMDAMRN', 'PDDUXLB', 'UICAMRH', 'OECBHSP', 'QLVIWOF', 'FEAILPN', 'TESMLDR', 'UURIFAE', 'QTTPXAD', 'WIAWLAC', 'CRPBHSR', 'ZIVQWOF', 'ROGTTNN', 'XEVIVIB', 'PDTTGAH', 'VIACLAY', 'KGJIEQH', 'GECMESN', 'NOCTHSG', 'GNVWTQH', 'KBPRHMV', 'UOYWLIA', 'FIRIGDB', 'OEBQLIG', 'WARQHNL', 'OISQKEP', 'EIDVXXN', 'ETPAXNZ', 'GDXWWEY', 'QCTIGON', 'NGJVHSQ', 'GEATHSY', 'GSDVVOA', 'QCXLHSP', 'QMDMETR', 'TMDUXTE', 'QQJMFAE', 'EAAIMEZ', 'REGIMUE', 'CICBXRV', 'QRSMENN', 'WTXTNSR', 'NBPZHMR', 'VRDYNEC', 'GSPMEAV', 'TENXKEQ', 'KCTTHSP', 'CMQQHSQ', 'GTXMFPB', 'GLWQZRB', 'OEIZHQH', 'GRTOBSG', 'TATTZRN', 'FOSMLED', 'WESIWDR', 'NAPBFOF', 'HEGIXLF', 'VOGUZLN', 'USRCRAZ', 'GZRTTAY', 'FEHKHMC', 'QNTZLEN', 'PUCKBAY', 'CICUBNR', 'PCXIWEY', 'CSIMFPR', 'UTPLXSY', 'CBGCCUY', 'CQJMWIE', 'KGTUBRH', 'VATTLEK', 'VACBXQH', 'GPDZEAN', 'NTJZTDR', 'NSDTFEV', 'PDXKTMV', 'NAIQMUQ', 'NOHKKOA', 'QMTBKOF', 'SUTUXPR', 'TMXBXNP', 'CLRCEAE', 'OIAWGGV', 'VUSGIOE', 'WLIQFOZ', 'KSPVMEB', 'LOHLXDV', 'CYSMGOP'

Agrupamos ahora por columnas, cada columna contiene las letras a las que se le suma la misma letra de la clave

In [434]:
def get_columns(text_blocks):
    group_size = len(text_blocks[0])
    columns = []
    for letter_count in range(group_size):
        column = ''
        for group_count in range(len(text_blocks)):
            column += text_blocks[group_count][letter_count]
        columns.append(column)
    return columns

In [435]:
columns=get_columns(blocks)
columns

['ULKQPUOQFTUQWCZRXPVKGNGKUFOWOEEGQNGGQQTQERCQWNVGTKCGGOGTFWNHVUGFQPCPCUCCKVVGNNPNNQSTCOVWKLCJUCWQPJVQKTWKCPVQWNPUQTGVKRKNKUEPUFWERIQUOWGHGQEGWKTKORKSPK',
 'EOTMDIELEEUTIRIOEDIGEONBOIEAIITDCGESCMMQAEIRTBRSECMTLERAOEAEOSZENUICSTBQGAAPTSDAOMUMLIULSOYEIRTSTOEELGPEGLRSANTONERRQONACEIAEREOOUVUENTESMILYOICBRDUTA',
 'CTPDDCCVASRTAPVGVTAJCCVPYRBRSDPXTJADXDDJAGCSXPDPNTQXWITTSSPGGRRHTCCXIPGJTTCDJDXIHTTXRASIPHSFGPPADGRAJTTSJDJWAPTBDEDDJCSHJMPJHPKGGYXCIBGRIJDPPCDPXDPTXE',
 'WTVAUABIIMIPWBQTITCIMTWRWIQQQVAWIVTVLMUMIIBMTZYMXTQMQZOTMIBIUCTKZKUIMLCMUTBZZTKQKBUBCWGQVLMCDMZWAQCVAIZMIAUIMDGMKCPAMLCXTQABMVMZCITUZIWBWVVOXMZLMNLAMI',
 'KVGMXMHWLLFXLHWTVGLEEHTHLGLHKXXWGHHVHEXFMMXENHNEKHHFZHBZLWFXZRTHLBBWFXCWBLXETFTMKKXXEGIFMXGXXLMLWSTTXEYELBXULXLXBLTTKXWTTZWBVMHBGBGTHGXHXBVNKQBTGNTXGK',
 'DAERLRSOPDAAASONIAAQSSQMIDINEXNEOSSOSTTAEURNSMEAESSPRQSREDOLLAAMEANEPSUIREQADEMUOOPNAGOOEDOMNCOPEONUMAIASNNIDGUSDEYQENARREELUERDANONEONPNCOAETNTLNLMEA',
 'VCZNBHPFNREDCRFNBHYHNGHVABGLPNZYNQYAPREEZEVNRRCVQPQBBHGNDRFFNZYCNYRY

In [436]:
# Desplaza una cadena sumandole la misma cantidad a cada caracter
def shift(text, amount):
    shifted = ''
    letters = string.ascii_uppercase
    for letter in text:
        shifted += letters[(letters.index(letter)+amount) % len(letters)]
    return shifted

In [437]:
def corr(a,b):
    s=0
    for letter in string.ascii_uppercase:
        s+=a.count(letter)*b.count(letter)
    return s/(len(a)*len(b))

In [438]:
mutIndCo={}
for i in range(keyLen):
    for j in range(i+1,keyLen):
        for sigma in range(26):
            mutIndCo[(i,j,sigma)]=corr(columns[i],shift(columns[j],sigma))

In [439]:
mutIndCo

{(0, 1, 0): 0.035111111111111114,
 (0, 1, 1): 0.03693333333333333,
 (0, 1, 2): 0.06848888888888889,
 (0, 1, 3): 0.038622222222222224,
 (0, 1, 4): 0.03546666666666667,
 (0, 1, 5): 0.03475555555555555,
 (0, 1, 6): 0.04391111111111111,
 (0, 1, 7): 0.030355555555555556,
 (0, 1, 8): 0.037333333333333336,
 (0, 1, 9): 0.036444444444444446,
 (0, 1, 10): 0.03542222222222222,
 (0, 1, 11): 0.03746666666666667,
 (0, 1, 12): 0.04351111111111111,
 (0, 1, 13): 0.03986666666666667,
 (0, 1, 14): 0.04008888888888889,
 (0, 1, 15): 0.03528888888888889,
 (0, 1, 16): 0.042977777777777774,
 (0, 1, 17): 0.03902222222222222,
 (0, 1, 18): 0.04195555555555556,
 (0, 1, 19): 0.03244444444444444,
 (0, 1, 20): 0.03591111111111111,
 (0, 1, 21): 0.028844444444444443,
 (0, 1, 22): 0.04208888888888889,
 (0, 1, 23): 0.029955555555555555,
 (0, 1, 24): 0.045733333333333334,
 (0, 1, 25): 0.032,
 (0, 2, 0): 0.03617777777777778,
 (0, 2, 1): 0.04448888888888889,
 (0, 2, 2): 0.037866666666666667,
 (0, 2, 3): 0.03875555555555556

In [445]:
mutIndCo_altos=list(filter(lambda x: mutIndCo[x] >= 0.065, mutIndCo))
mutIndCo_altos

[(0, 1, 2),
 (0, 2, 13),
 (0, 3, 20),
 (0, 4, 9),
 (0, 5, 2),
 (0, 6, 15),
 (1, 2, 11),
 (1, 3, 18),
 (1, 4, 7),
 (1, 5, 0),
 (1, 6, 13),
 (2, 3, 7),
 (2, 4, 22),
 (2, 5, 15),
 (2, 6, 2),
 (3, 4, 15),
 (3, 5, 8),
 (3, 6, 21),
 (4, 5, 19),
 (4, 6, 6),
 (5, 6, 13)]

In [446]:
possible_keys=[]
for first in string.ascii_uppercase:
    key=first
    for i in mutIndCo_altos[:keyLen-1]:
        key+=shift(first,i[2])
    possible_keys.append(key)
possible_keys

['ACNUJCP',
 'BDOVKDQ',
 'CEPWLER',
 'DFQXMFS',
 'EGRYNGT',
 'FHSZOHU',
 'GITAPIV',
 'HJUBQJW',
 'IKVCRKX',
 'JLWDSLY',
 'KMXETMZ',
 'LNYFUNA',
 'MOZGVOB',
 'NPAHWPC',
 'OQBIXQD',
 'PRCJYRE',
 'QSDKZSF',
 'RTELATG',
 'SUFMBUH',
 'TVGNCVI',
 'UWHODWJ',
 'VXIPEXK',
 'WYJQFYL',
 'XZKRGZM',
 'YALSHAN',
 'ZBMTIBO']

In [444]:
for key in possible_keys:
    vig=Vigenere(string.ascii_uppercase,key)
    print(vig.decipher(text[:50]))

UCPCBBGLMGZMYNKRCBXCKQKQGDPYPBQAOJMUGPGDPSOCPHYQAQ
TBOBAAFKLFYLXMJQBAWBJPJPFCOXOAPZNILTFOFCORNBOGXPZP
SANAZZEJKEXKWLIPAZVAIOIOEBNWNZOYMHKSENEBNQMANFWOYO
RZMZYYDIJDWJVKHOZYUZHNHNDAMVMYNXLGJRDMDAMPLZMEVNXN
QYLYXXCHICVIUJGNYXTYGMGMCZLULXMWKFIQCLCZLOKYLDUMWM
PXKXWWBGHBUHTIFMXWSXFLFLBYKTKWLVJEHPBKBYKNJXKCTLVL
OWJWVVAFGATGSHELWVRWEKEKAXJSJVKUIDGOAJAXJMIWJBSKUK
NVIVUUZEFZSFRGDKVUQVDJDJZWIRIUJTHCFNZIZWILHVIARJTJ
MUHUTTYDEYREQFCJUTPUCICIYVHQHTISGBEMYHYVHKGUHZQISI
LTGTSSXCDXQDPEBITSOTBHBHXUGPGSHRFADLXGXUGJFTGYPHRH
KSFSRRWBCWPCODAHSRNSAGAGWTFOFRGQEZCKWFWTFIESFXOGQG
JRERQQVABVOBNCZGRQMRZFZFVSENEQFPDYBJVEVSEHDREWNFPF
IQDQPPUZAUNAMBYFQPLQYEYEURDMDPEOCXAIUDURDGCQDVMEOE
HPCPOOTYZTMZLAXEPOKPXDXDTQCLCODNBWZHTCTQCFBPCULDND
GOBONNSXYSLYKZWDONJOWCWCSPBKBNCMAVYGSBSPBEAOBTKCMC
FNANMMRWXRKXJYVCNMINVBVBROAJAMBLZUXFRAROADZNASJBLB
EMZMLLQVWQJWIXUBMLHMUAUAQNZIZLAKYTWEQZQNZCYMZRIAKA
DLYLKKPUVPIVHWTALKGLTZTZPMYHYKZJXSVDPYPMYBXLYQHZJZ
CKXKJJOTUOHUGVSZKJFKSYSYOLXGXJYIWRUCOXOLXAWKXPGYIY
BJWJIINSTNGTFURYJIEJRXRXNKWFWIX